Learning to use decorator, getters, setters, properties. 

This uses the performance timing methods in the functools wraps module to create a @timeit decorator you can stick in front of any function to also return the time it took to run.

In [ ]:
from functools import wraps
import time

def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f"Function {func.__name__}{args} {kwargs} took {total_time:.4f} seconds")
        return result
    return timeit_wrapper

In [4]:
from dataclasses import dataclass
from datetime import datetime

@dataclass
class Transaction:
    _sender: str
    _recipient: str
    _amount: float
    _date: datetime = None

    # Sender getter and setter

    @property
    def sender(self):
        return self._sender
    
    @sender.setter
    def sender(self, sender):
        if not isinstance(sender, str):
            raise TypeError("Sender must be a string")
        self._sender = sender

    # Recipient getter and setter

    @property
    def recipient(self):
        return self._recipient

    @recipient.setter
    def recipient(self, recipient):
        if not isinstance(recipient, str):
            raise TypeError("Recipient must be a string")
        self._recipient = recipient

    # Amount getter and setter

    @property
    def amount(self):
        return self._amount

    @amount.setter
    def amount(self, amount):
        if not isinstance(amount, float):
            raise TypeError("Amount must be a float")
        self._amount = amount

    # Date getter and setter and deleter

    @property
    def date(self):
        return self._date

    @date.setter
    def date(self, date):
        if date is not None and not isinstance(date, str):
            raise TypeError("Date must be a string or None")
        self._date = date

    @date.deleter
    def date(self):
        self._date = None




Now with pydantic: 

In [2]:
pip install pydantic

  Using cached pydantic-2.0.2-py3-none-any.whl (359 kB)
  Using cached pydantic_core-2.1.2-cp310-cp310-macosx_10_7_x86_64.whl (1.5 MB)
  Using cached annotated_types-0.5.0-py3-none-any.whl (11 kB)
  Using cached typing_extensions-4.7.1-py3-none-any.whl (33 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In pydantic, you don't usually use getters and setters, the validation happens when the object is instantiated. So here we weill build validators instead of getters and setters

In [3]:
from pydantic import BaseModel, Field, validator, ValidationError

class TransactionPydantic(BaseModel):
    sender: str
    recipient: str
    _amount: float
    _date: datetime | None # this is a type hint for python 3.10  if using 3.9 you have to Optional from typing

@validator("sender", "recipient")
def check_string(cls, v):
    if not isinstance(v, str):
        raise TypeError("Sender and recipient must be strings")
    return v

@validator("amount")
def check_amount(cls, v):
    if not isinstance(v, float):
        raise TypeError("Amount must be a float")
    return v

@validator("date")
def check_date(cls, v):
    if v is not None and not isinstance(v, datetime):
        raise TypeError("Date must be datetime or None")
    return v
